In [1]:
# Collect Data
'''
Records screen from the game and saves it to a file
- GTA V 
- Windowed mode
- Resolution: 800 x 600
- Keep window at top left corner of screen (0,0)
'''

''

In [2]:
# python 3.7.9
import numpy as np
print(np.__version__) #1.18.5

1.18.5


In [4]:
import os
import time
import cv2
import numpy as np
from grabscreen import grab_screen
from getkeys import key_check

key_map = {
    'W': [1, 0, 0, 0, 0, 0, 0, 0, 0],
    'S': [0, 1, 0, 0, 0, 0, 0, 0, 0],
    'A': [0, 0, 1, 0, 0, 0, 0, 0, 0],
    'D': [0, 0, 0, 1, 0, 0, 0, 0, 0],
    'WS': [0, 0, 0, 0, 1, 0, 0, 0, 0],
    'WD': [0, 0, 0, 0, 0, 1, 0, 0, 0],
    'SA': [0, 0, 0, 0, 0, 0, 1, 0, 0],
    'SD': [0, 0, 0, 0, 0, 0, 0, 1, 0],
    'NK': [0, 0, 0, 0, 0, 0, 0, 0, 1],
    'default': [0, 0, 0, 0, 0, 0, 0, 0, 0],
}
# key map for the output

starting_value = 1
BATCH_SIZE = 1000 #~370Mb/1000 images
mode = 'RGB' # 'RGB' or 'GRAY'

while True:
    file_name = 'training_data-{0}.npy'.format(starting_value)
    if os.path.isfile(file_name):
        print('File exists, moving along', starting_value)
        starting_value += 1
    else:
        print('File does not exist, starting fresh!', starting_value)
        break

def keys_to_output(keys):
    if ''.join(keys) in key_map:
        return key_map[''.join(keys)]
    return key_map['default']

def main(file_name, starting_value, mode BATCH_SIZE):
    training_data = []
    for i in range(4, 0, -1):
        print(i + 1)
        time.sleep(1)
    paused = False
    print('STARTING!!!')
    while True:
        if not paused:
            screen = grab_screen(region=(0, 40, 800, 640))
            # region defines the area of the screen to capture

            screen = cv2.resize(screen, (480, 270))
            # resize the screen to 480x270

            mode = cv2.COLOR_BGR2RGB if mode == 'RGB' else cv2.COLOR_BGR2GRAY
            screen = cv2.cvtColor(screen, mode)
            # convert the screen from BGR to color mode

            keys = key_check()
            # get the keys pressed

            output = keys_to_output(keys)
            training_data.append([screen, output])
            # append the screen and the output to the training data

            if len(training_data) % 100 == 0:
                print(len(training_data))
                if len(training_data) == BATCH_SIZE:
                    np.save(file_name, np.array(training_data))
                    print('SAVED')
                    training_data = []
                    starting_value += 1
                    file_name = 'training_data-{0}.npy'.format(starting_value)

        keys = key_check()
        if 'T' in keys:
            if paused:
                paused = False
                print('Unpaused!')
                time.sleep(1)
            else:
                print('Pausing!')
                paused = True
                time.sleep(1)

if __name__ == "__main__":
    main(file_name, starting_value, mode, BATCH_SIZE)

File does not exist, starting fresh! 1
4
3
2
1
STARTING!!!
100
200
300
400
500
600
700
800
900
1000
SAVED
100
200
300
400
500
600
700
800
900
1000
SAVED
100
200
300
400
500
600
700
Pausing!
